In [1]:
import requests
import pattern3.web as web
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from progressbar import ProgressBar
from bs4 import BeautifulSoup

<br><br/> 

In [84]:
class build_dataset(object):
    
    def __init__(self, terms=None):
        
        self.names_ = []
        self.genres_ = [] 
        self.genreIDs_ = []
        self.IDs_desc_ = []
        self.terms = terms 
        
        
        


    def pull_genres(self, my_data):
        

        all_genres = [] 
        all_genreIDs = []
        all_genreswithIDs = [] 
        new_list_IDs = []
        urls = [] 
        rss_feeds = [] 
        names = []
        columns= ['Name', 'Artwork', 'Episode Count', 'GenreIDs', 'iTunes URL', 'FeedURL'] 

        
        pbar = ProgressBar()
        
        
        
        
        if my_data["resultCount"] != 0: 
            for elem in pbar(my_data["results"]):   
                if "kind" in elem.keys(): 
                    if elem["kind"] == "podcast":
                        
                        if "trackCount" in elem.keys():
                            if elem["trackCount"] >= 20: 
                                
                                episode_count = elem["trackCount"]
                                
                                if "genreIds" in elem.keys():
                                    genreID = elem["genreIds"]
                                    all_genreIDs.append(genreID)

                                if "genres" in elem.keys():
                                    genre = elem["genres"] 
                                    all_genres.append(genre)
                                    genresandIDs = list(zip(genreID, genre))
                                    all_genreswithIDs.append(genresandIDs)

                                if "collectionName" in elem.keys():
                                    collection_name = elem["collectionName"]
                                    names.append(collection_name)


                                if "collectionViewUrl" in elem.keys(): 
                                    itunes_url = elem["collectionViewUrl"]
                                                                                           
                                elif "trackViewUrl" in elem.keys():
                                    itunes_url = elem["trackViewUrl"]
                                    

                                if "feedUrl" in elem.keys():
                                    feedurl = elem["feedUrl"] 
                                    
                                    
                                if "artworkUrl100" in elem.keys():
                                    artwork = elem["artworkUrl100"]
                                    
                                


                                my_row = [collection_name, artwork, episode_count, genreID, itunes_url, feedurl]
                                my_row_pd = pd.DataFrame([my_row], columns = columns)                    
                                self.df_podcast_ = self.df_podcast_.append(my_row_pd, ignore_index=True)



                                self.genres_ = list(set(np.concatenate(all_genres).tolist()))
                                self.genreIDs_ = list(set(np.concatenate(all_genreIDs).tolist()))
                                self.names_ = names  
                                new_list_IDs = np.concatenate(all_genreswithIDs).tolist()
                                self.IDs_desc_ = [ii for n,ii in enumerate(new_list_IDs) if ii not in new_list_IDs[:n]]
                        
                        else:
                            pass 


                    else:
                        pass 

                else:
                    pass 
                   
        else: 
            pass 
        
        return self
        

    
    def build_random_data(self, term):
        
        
        url = "https://itunes.apple.com/search?term=" + term + "&limit=200&country=US&lan=en_us&entity=podcast"
        
        try:
            response = requests.get(url) 
            data = response.json()

            print (data["resultCount"])
            if data["resultCount"] != 0: 
                self.pull_genres(data) 
                return self  
            else: 
                return self
        
        except ValueError:
                pass 



    def build_genres_data(self, genre):
        
        url = "https://itunes.apple.com/search?term=" + genre + "&limit=200&country=US&lan=en_us&entity=podcast"
        
        try: 
            response = requests.get(url) 
            data = response.json()

            if data["resultCount"] != 0: 
                self.pull_genres(data) 
                return self  
            else: 
                return self
        
        except ValueError:
                pass 


        
        
    def find_genres(self, gnr):
        
        self.build_genres_data(gnr)

        genres_list = self.genres_
        names_list = self.names_
        IDs_genres_list = self.IDs_desc_

        if names_list != []: 
            for name in names_list:
                if name not in self.initial_names_:
                    self.initial_names_.append(name)
        
        if IDs_genres_list != []:
            for item in IDs_genres_list:
                if item not in self.genres_with_IDs_:
                    self.genres_with_IDs_.append(item)

        if genres_list != []: 
            for genre in genres_list:
                if genre not in self.initial_genres_:
                    self.initial_genres_.append(genre)
                    print ("extra")
                    self.find_genres(genre)
                  
        return self 
                    
        

    def initialize(self):
        self.initial_genres_ = None
        self.genreIDs_ = None
        self.names_ = None 
        self.genres_with_IDs_ = None
        
        url = "https://itunes.apple.com/search?term=podcasts&limit=200&country=US&lan=en_us&entity=podcast" 
        
        
        columns= ['Name', 'Artwork', 'Episode Count', 'GenreIDs', 'iTunes URL', 'FeedURL']
        self.df_podcast_ = pd.DataFrame([[0, 0, 0, 0, 0, 0]], columns = columns)

        try: 
            response = requests.get(url) 
            my_data = response.json()

            if my_data["resultCount"] != 0:
                self.pull_genres(my_data)

                self.initial_genres_ = self.genres_
                self.initial_genreIDs_ = self.genreIDs_
                self.genres_with_IDs_ = self.IDs_desc_
                self.initial_names_ = self.names_
                

            else: 
                pass 
            
        except ValueError:
            print ("Decoding JSON has failed")
        

                    
    def build_data(self):
        
        terms = self.terms
        self.initialize()
        
        if self.initial_genres_ is not None:
            for initial_genre in self.initial_genres_: 
                self.find_genres(initial_genre)
        
        print (terms)
        if terms is not None:
            for term in terms:
                self.build_random_data(term)
        

        return self 
        

In [85]:
more_genres = ["random", "editorial", "review", "hobbies", "fiction", "mystery", "adventure", "romance", "humor"]

In [86]:
data_builder = build_dataset(more_genres)

In [87]:
data_builder.build_data()

100% |########################################################################|
100% |########################################################################|


extra


100% |########################################################################|
100% |########################################################################|


extra


100% |########################################################################|
100% |########################################################################|


...

200


76



100% |########################################################################|


114


100% |########################################################################|


In [ ]:
genre_IDs = data_builder.genres_with_IDs_

with open('genre_IDs.txt','a') as t:
    t.write('\n' + str(genre_IDs))      
    t.close()

In [89]:
podcast_dataframe = data_builder.df_podcast_.drop_duplicates(['Name']).reset_index(drop=True)

In [90]:
podcast_dataframe.drop(podcast_dataframe.index[0], inplace=True)

In [91]:
podcast_dataframe = podcast_dataframe.reset_index(drop=True)

In [111]:
podcast_dataframe.to_csv("df_01.csv", index=False)

In [92]:
podcast_dataframe

,Name,Artwork,Episode Count,GenreIDs,iTunes URL,FeedURL
0,Real Time with Bill Maher,http://is4.mzstatic.com/image/thumb/Music62/v4...,251,"[1311, 26]",https://itunes.apple.com/us/podcast/real-time-...,http://billmaher.hbo.libsynpro.com/rss
1,Sword and Scale,http://is2.mzstatic.com/image/thumb/Music71/v4...,99,"[1479, 26, 1315, 1324, 1302, 1311]",https://itunes.apple.com/us/podcast/sword-and-...,https://rss.art19.com/sword-and-scale
2,On Point with Tom Ashbrook | Podcasts,http://is3.mzstatic.com/image/thumb/Music128/v...,50,"[1311, 26]",https://itunes.apple.com/us/podcast/on-point-w...,https://www.npr.org/rss/podcast.php?id=510053
3,Learn French with daily podcasts,http://is1.mzstatic.com/image/thumb/Music71/v4...,291,"[1469, 26, 1304]",https://itunes.apple.com/us/podcast/learn-fren...,http://www.dailyfrenchpod.com/wordpress/new-rs...
4,Learn Italian with Podcasts at LearnItalianPod...,http://is1.mzstatic.com/image/thumb/Music/v4/a...,175,"[1469, 26, 1304, 1324, 1320, 1416]",https://itunes.apple.com/us/podcast/learn-ital...,http://www.learnitalianpod.com/feed/
5,Road Trippin' with RJ & Channing,http://is5.mzstatic.com/image/thumb/Music111/v...,41,"[1465, 26, 1316, 1324]",https://itunes.apple.com/us/podcast/road-tripp...,https://rss.art19.com/road-trippin-with-rj-and...
6,ESPN Podcasts,http://is2.mzstatic.com/image/thumb/Music71/v4...,30,"[1316, 26]",https://itunes.apple.com/us/podcast/espn-podca...,http://www.espn.com/espnradio/podcast/feeds/it...
7,Wrongful Conviction with Jason Flom,http://is3.mzstatic.com/image/thumb/Music111/v...,36,"[1324, 26]",https://itunes.apple.com/us/podcast/wrongful-c...,https://rss.art19.com/wrongful-conviction-with...
8,Podcasts – The Mike O'Meara Show,http://is4.mzstatic.com/image/thumb/Music71/v4...,40,"[1303, 26, 1324, 1311]",https://itunes.apple.com/us/podcast/podcasts-t...,http://mikeomearashow.com/category/podcasts/feed/
9,Slate's Double X Podcasts,http://is1.mzstatic.com/image/thumb/Music62/v4...,144,"[1324, 26, 1305]",https://itunes.apple.com/us/podcast/slates-dou...,http://feeds.feedburner.com/DoubleXPodcasts
